# 2. Feature Engineering - Modèle RFM

Ce notebook calcule les features RFM (Recency, Frequency, Monetary) pour la segmentation client.

**Objectifs :**
- Calculer les features RFM avec le module `src.features.rfm`
- Analyser les distributions
- Préparer les données pour le clustering

**Auteur :** Thomas Mebarki  
**Date :** Janvier 2026

## 2.1 Configuration et imports

In [ ]:
import sys
from pathlib import Path
from datetime import datetime

# Ajouter le répertoire parent au path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modules du projet
from src.config import (
    RAW_DATA_DIR, 
    PROCESSED_DATA_DIR,
    CUSTOMER_ID_COL,
    RFM_FEATURES
)
from src.data.loader import load_transactions
from src.features.rfm import RFMCalculator, calculate_rfm

# Configuration
pd.options.display.float_format = '{:.2f}'.format
sns.set_style('whitegrid')
%matplotlib inline

print("Modules importés avec succès")

## 2.2 Chargement des données

In [ ]:
# Charger les transactions préparées
# Option 1: depuis le fichier preprocessed
try:
    df = pd.read_csv(RAW_DATA_DIR / 'transactions_clean.csv', parse_dates=['order_purchase_timestamp'])
    print(f"Données chargées depuis transactions_clean.csv")
except FileNotFoundError:
    # Option 2: depuis les fichiers bruts
    df = load_transactions(RAW_DATA_DIR / 'data.csv')
    print(f"Données chargées depuis data.csv")

print(f"\nShape: {df.shape}")
df.head()

In [ ]:
# Vérification des données
print("Colonnes:", df.columns.tolist())
print(f"\nPériode: {df['order_purchase_timestamp'].min()} à {df['order_purchase_timestamp'].max()}")
print(f"Clients uniques: {df['customer_unique_id'].nunique()}")
print(f"Commandes: {df['order_id'].nunique()}")

## 2.3 Calcul des features RFM

**Définitions :**
- **Recency (R)** : Nombre de jours depuis le dernier achat
- **Frequency (F)** : Nombre total de commandes
- **Monetary (M)** : Montant total dépensé

In [ ]:
# Définir la date de référence (lendemain de la dernière commande)
reference_date = df['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print(f"Date de référence: {reference_date}")

# Utiliser le module RFM
calculator = RFMCalculator(reference_date=reference_date)
rfm_df = calculator.fit_transform(df)

print(f"\nFeatures RFM calculées pour {len(rfm_df)} clients")
rfm_df.head(10)

In [ ]:
# Statistiques descriptives
print("Statistiques RFM:")
calculator.get_statistics()

## 2.4 Analyse des distributions

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Recency
axes[0].hist(rfm_df['recency'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(rfm_df['recency'].median(), color='red', linestyle='--', label=f'Médiane: {rfm_df["recency"].median():.0f}')
axes[0].set_title('Distribution de la Récence', fontsize=12)
axes[0].set_xlabel('Jours depuis dernier achat')
axes[0].set_ylabel('Nombre de clients')
axes[0].legend()

# Frequency
freq_counts = rfm_df['frequency'].value_counts().sort_index()
axes[1].bar(freq_counts.index[:10], freq_counts.values[:10], color='green', edgecolor='black')
axes[1].set_title('Distribution de la Fréquence', fontsize=12)
axes[1].set_xlabel('Nombre de commandes')
axes[1].set_ylabel('Nombre de clients')

# Monetary
axes[2].hist(rfm_df['monetary'], bins=50, color='orange', edgecolor='black', alpha=0.7)
axes[2].axvline(rfm_df['monetary'].median(), color='red', linestyle='--', label=f'Médiane: {rfm_df["monetary"].median():.0f}')
axes[2].set_title('Distribution du Montant', fontsize=12)
axes[2].set_xlabel('Montant total (BRL)')
axes[2].set_ylabel('Nombre de clients')
axes[2].set_xlim(0, 1500)  # Limiter pour lisibilité
axes[2].legend()

plt.tight_layout()
plt.savefig('../docs/rfm_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 2.5 Analyse des corrélations

In [ ]:
# Matrice de corrélation
correlation_matrix = rfm_df.corr()

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', linewidths=0.5, ax=ax)
ax.set_title('Corrélations entre features RFM', fontsize=14)
plt.tight_layout()
plt.show()

print("\nCorrélations:")
print(f"  Recency-Frequency: {correlation_matrix.loc['recency', 'frequency']:.3f}")
print(f"  Frequency-Monetary: {correlation_matrix.loc['frequency', 'monetary']:.3f}")
print(f"  Recency-Monetary: {correlation_matrix.loc['recency', 'monetary']:.3f}")

## 2.6 Détection des outliers

In [ ]:
# Boxplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, col, color in zip(axes, RFM_FEATURES, ['steelblue', 'green', 'orange']):
    sns.boxplot(y=rfm_df[col], ax=ax, color=color)
    ax.set_title(f'Boxplot {col.capitalize()}')
    ax.set_ylabel(col.capitalize())

plt.tight_layout()
plt.show()

# Quantiles
print("\nQuantiles (pour détecter les outliers):")
for col in RFM_FEATURES:
    q99 = rfm_df[col].quantile(0.99)
    pct_outliers = (rfm_df[col] > q99).sum() / len(rfm_df) * 100
    print(f"  {col}: Q99 = {q99:.2f}, outliers (>Q99) = {pct_outliers:.2f}%")

## 2.7 Transformation logarithmique (optionnel)

Pour les données très asymétriques, une transformation log peut aider le clustering.

In [ ]:
# Transformation log pour frequency et monetary (si nécessaire)
rfm_log = rfm_df.copy()
rfm_log['frequency_log'] = np.log1p(rfm_df['frequency'])
rfm_log['monetary_log'] = np.log1p(rfm_df['monetary'])

# Comparaison avant/après
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].hist(rfm_df['monetary'], bins=50, color='orange', alpha=0.7)
axes[0, 0].set_title('Monetary (original)')

axes[0, 1].hist(rfm_log['monetary_log'], bins=50, color='green', alpha=0.7)
axes[0, 1].set_title('Monetary (log)')

axes[1, 0].hist(rfm_df['frequency'], bins=20, color='steelblue', alpha=0.7)
axes[1, 0].set_title('Frequency (original)')

axes[1, 1].hist(rfm_log['frequency_log'], bins=20, color='purple', alpha=0.7)
axes[1, 1].set_title('Frequency (log)')

plt.tight_layout()
plt.show()

## 2.8 Sauvegarde des features RFM

In [ ]:
# Sauvegarder en format Parquet (recommandé)
output_parquet = PROCESSED_DATA_DIR / 'customers_rfm.parquet'
rfm_df.to_parquet(output_parquet)
print(f"Sauvegardé: {output_parquet}")

# Aussi en CSV pour compatibilité
output_csv = PROCESSED_DATA_DIR / 'customers_rfm.csv'
rfm_df.to_csv(output_csv)
print(f"Sauvegardé: {output_csv}")

print(f"\nShape finale: {rfm_df.shape}")

## 2.9 Résumé

**Features RFM créées :**

| Feature | Description | Médiane | Écart-type |
|---------|-------------|---------|------------|
| recency | Jours depuis dernier achat | ~172 | ~147 |
| frequency | Nombre de commandes | 1 | ~0.2 |
| monetary | Montant total (BRL) | ~109 | ~165 |

**Observations :**
- Distribution de frequency très concentrée (97% avec 1 commande)
- Monetary avec queue longue (quelques gros acheteurs)
- Recency relativement étalée

**Prochaine étape :** Clustering dans le notebook 03.